In [ ]:
import os

from veil.veil_client import *
from veil.constants import NetworkId, NETWORK_INFO
from utils.logutils import setup_logger
import logging

LOGGER = setup_logger("VeilPlayground")
LOGGER = setup_logger("veil.veil_client", log_level=logging.INFO)

NETWORK_ID = NetworkId.KOVAN
PRIVATE_KEY = os.environ.get("PRIVATE_KEY")
INFURA_API_KEY = os.environ.get("INFURA_API_KEY")
WEB3_RPC_URL = "https://{}.infura.io/{}".format(NETWORK_ID.name.lower(), INFURA_API_KEY)

client = VeilClient(
    network_id=NETWORK_ID.value,
    web3_rpc_url=WEB3_RPC_URL,
    private_key=PRIVATE_KEY,
)

MY_ADDRESS = client.account_address

print(
    "NETWORK:",  NETWORK_ID,
    "\nRPC_URL", WEB3_RPC_URL,
    "\nADDRESS IN USE:", MY_ADDRESS,
    "\nETH BALANCE:", client.get_eth_balance()
)

print(client)
print("Signing into veil...")
print(client.authenticate())

In [ ]:
client.get_eth_balance()

In [ ]:
markets = client.get_markets(
#     channel="btc", 
    status="open",
#     page=0
#     raw_json=True,
)
print(len(markets))
markets[0]

In [ ]:
from pprint import pprint
print(set([m.type for m in markets]))
print(set([m.channel for m in markets]))
print([(m.type.value, m.num_ticks) for m in markets])

In [ ]:
market = client.get_market(
    "veil-market-creation",
#     raw_json=True
)
market

In [ ]:
from veil.veil_client import TokenType, OrderSide

bids_long = client.get_bids(
    market_slug=market.slug,
    token_type=TokenType.LONG,
)
bids_short = client.get_bids(
    market_slug=market.slug,
    token_type=TokenType.SHORT,
)
asks_long = client.get_asks(
    market_slug=market.slug,
    token_type=TokenType.LONG,
)
asks_short = client.get_asks(
    market_slug=market.slug,
    token_type=TokenType.SHORT,
)
bids_long, bids_short, asks_long, asks_short

In [ ]:
print(eth_price_from_veil_price('5000', market.num_ticks))
print(amount_from_veil_shares('5000749962502', market.num_ticks))

In [ ]:
order_fills = client.get_order_fills(
    market_slug=market.slug,
    token_type=TokenType.SHORT,
#     raw_json=True
)
order_fills

In [ ]:
from veil.veil_client import OrderStatus

client.get_my_orders(
    market.slug,
#     order_status=None
#     raw_json=True
)

In [ ]:
from pprint import pprint
quote = client.get_quote(
    market=market,
    token_type="long",
    side="buy",
    amount=0.01,
    price=0.1,
)
zx_order = quote.zero_ex_order
assert quote.order_hash == zx_order.hash
quote

In [ ]:
zx_order.signature = client.sign_hash_0x_compat(zx_order.hash)
zx_order